In [1]:
import pandas as pd
import numpy as np

import quandl

import matplotlib.pyplot as plt

import cvxpy as cp

%matplotlib inline

In [2]:
quandl.api_config.ApiConfig.api_key = 'Qb1phPtM_beaDTohoBYK'

In [3]:
tkrs=[
    'AAPL',
    'MSFT',
    'WMT',
    'AMZN',
    'BA',
    'NKE',
    'GILD',
    'TJX',
    'AMAT',
    'PSX'

]

In [4]:
data = quandl.get_table('WIKI/PRICES', ticker = tkrs, 
                        qopts = { 'columns': ['ticker', 'date', 'adj_close'] }, 
                        date = { 'gte': '2016-12-31', 'lte': '2019-12-31' }, 
                        paginate=True)

data = data.pivot_table(values='adj_close',index='date',columns='ticker')

In [5]:
returns = data.pct_change()
returns.dropna(inplace=True)

In [6]:
returns.tail()

ticker,AAPL,AMAT,AMZN,BA,GILD,MSFT,NKE,PSX,TJX,WMT
date,,,,,,,,,,
2018-03-21,-0.022655,0.026169,-0.002931,-0.001570,-0.005590,-0.006979,-0.006737,0.011041,0.012708,0.002615
2018-03-22,-0.014159,-0.030602,-0.023871,-0.051884,-0.012028,-0.029087,-0.029088,-0.015081,-0.022683,-0.011794
2018-03-23,-0.023128,-0.060081,-0.031436,0.004349,-0.036125,-0.029068,0.003260,-0.013833,-0.013827,-0.019738
2018-03-26,0.047472,0.055796,0.040319,0.024829,0.037068,0.075705,0.019650,0.027947,0.014897,0.024350
2018-03-27,-0.025641,-0.051821,-0.037799,-0.023862,-0.018930,-0.045959,0.004097,-0.003021,-0.007771,-0.016571


In [7]:
totalreturns = returns.mean().values

In [8]:
Sigma = returns.cov()

In [9]:
x1 = cp.Variable(10)
risk= cp.quad_form(x1,Sigma)
constraints = [cp.sum(x1) == 1]

In [10]:
problem = cp.Problem(cp.Minimize(risk),constraints)

In [11]:
problem.solve()

4.812298206681895e-05

In [12]:
x1.value

array([ 0.1235465 , -0.05473012,  0.13766268,  0.06280058,  0.08248954,
        0.00029167,  0.09812001,  0.21484009,  0.20300432,  0.13197473])

In [13]:
np.dot(returns,(x1.value)).var()

4.7966738618550065e-05

In [14]:
x1.value.T.dot(Sigma).dot(x1.value)

4.812298206681895e-05

In [69]:
wts0=np.random.uniform(-100,100,len(tkrs))

C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [70]:
wts0

C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([-61.97302243,  -5.23191862, -32.58588554, -11.19104882,
        70.23822968,  25.76020678, -38.16722299,  31.68832645,
       -55.14411194, -30.24914805])

In [71]:
wts0.T.dot(Sigma).dot(wts0)

C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2.698925187420609

In [72]:
x2 = cp.Variable(10)
risk2= cp.quad_form((x2+wts0),Sigma)
constraints2 = [x2>=-1,x2<=1]

C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [73]:
problem2 = cp.Problem(cp.Minimize(risk2),constraints2)

C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [74]:
problem2.solve()

C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2.5555784445329186

In [75]:
x2.value

C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([ 1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.])

In [76]:
(x2.value+wts0).T.dot(Sigma).dot(x2.value+wts0)

C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2.5555784445329186

In [77]:
totalreturns

C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([0.00132311, 0.00200238, 0.00233206, 0.00251424, 0.00017991,
       0.0013046 , 0.00091848, 0.00045377, 0.00031557, 0.00088688])

In [78]:
#minimize risk s.t. return =0
x3 = cp.Variable(10)
ret = totalreturns.T*(wts0+x3)

risk3= cp.quad_form((x3+wts0),Sigma)
constraints3 =[ret==0]
problem3 = cp.Problem(cp.Minimize(risk3),constraints3)

C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\mattk\anaconda3\lib\site-packages\cvxpy\expressions\expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


In [79]:
problem3.solve()

C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


3.0425549374984234e-16

In [80]:
x3.value

C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([ 61.97302163,   5.23191832,  32.58588497,  11.19104894,
       -70.23822991, -25.76020464,  38.167223  , -31.68832658,
        55.14411149,  30.2491482 ])

In [81]:
wts0

C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([-61.97302243,  -5.23191862, -32.58588554, -11.19104882,
        70.23822968,  25.76020678, -38.16722299,  31.68832645,
       -55.14411194, -30.24914805])

In [82]:
#minimize risk w/ L1 penalty

C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [86]:
lam = .02
x4 = cp.Variable(10)

risk4= cp.quad_form((x4+wts0),Sigma) + lam * cp.norm(x4,1)
constraints4 =[]
problem4 = cp.Problem(cp.Minimize(risk4),constraints4)

C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [87]:
problem4.solve(verbose=True,solver='OSQP')

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 30, constraints m = 30
          nnz(P) + nnz(A) = 115
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -1.6000e+00   7.02e+01   1.40e+02   1.00e-01   1.16e-04s
  50   2.6857e+00   1.96e-06   2.11e-09   1.08e-02   2.99e-04s
plsh   2.6857e+00   7.95e-19   7.05e-17   -----

C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2.685725005014192

In [88]:
print(x4.value)
print('var after: {}'.format((x4.value+wts0).T.dot(Sigma).dot(x4.value+wts0)))
print('Total trades: {}'.format((np.absolute(x4.value) > 1e-8).sum()))

[ 6.23295225e+00  7.95079972e-19  5.87920056e-19  1.89478239e-19
 -2.25361635e+00  4.27547731e-19  5.10232781e+00  1.01058666e-19
 -7.46138171e-20  8.59453588e-20]
var after: 2.4139470768718736
Total trades: 3


C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [48]:
x4.value

C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([ 1.35273971e-22, -3.79700920e+00, -4.09578254e+00, -1.07835553e-24,
        2.36664031e+00,  6.89566549e-23,  4.05162729e-23, -3.13706451e-23,
       -4.24387474e-23,  2.11125076e-24])

In [89]:
print('var before: {}'.format((wts0).T.dot(Sigma).dot(wts0)))

var before: 2.698925187420609


C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [90]:
print('var after: {}'.format((x4.value+wts0).T.dot(Sigma).dot(x4.value+wts0)))
print('Total trades: {}'.format((np.absolute(x4.value) > 1e-8).sum()))

var after: 2.4139470768718736
Total trades: 3


C:\Users\mattk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
